In [ ]:
# API를 코드 밖에서 관리하게 함
# !pip install python-dotenv

In [ ]:
# XML을 dict로 바꿔줌
# !pip install xmltodict

In [ ]:
# nbstripout 설치
# !pip install nbstripout

# 특허·실용 공개·등록공보
- getAdvancedSearch

In [ ]:
import os
import requests
import xmltodict
import pandas as pd
from dotenv import load_dotenv
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ===============================
# 1. 환경변수 로드
# ===============================
load_dotenv()
API_KEY = os.getenv("KIPRIS_API_KEY")

if not API_KEY:
    raise RuntimeError(
        "[ERROR] 환경변수 로드 실패\n"
        "- 원인: KIPRIS_API_KEY가 설정되어 있지 않습니다.\n"
        "- 확인사항: .env 파일에 KIPRIS_API_KEY가 정의되어 있는지 확인하세요."
    )

# ===============================
# 2. Endpoint
# ===============================
BASE_URL = (
    "http://plus.kipris.or.kr/kipo-api/kipi/"
    "patUtiModInfoSearchSevice/getAdvancedSearch"
)

# ===============================
# 3. 검색 조건
# ===============================
COMPANY_NAME = "터보파워텍"

params = {
    "ServiceKey": API_KEY,
    "applicant": COMPANY_NAME,
    "patent": "true",
    "utility": "true",
    "numOfRows": 500,
    "pageNo": 1
}

# ===============================
# 4. requests Session + Retry 설정
# ===============================
session = requests.Session()

retry_strategy = Retry(
    total=3,                     # 최대 재시도 횟수
    backoff_factor=1,             # 1s, 2s, 4s 대기
    status_forcelist=[500, 502, 503, 504],
    allowed_methods=["GET"]
)

adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("http://", adapter)
session.mount("https://", adapter)

# ===============================
# 5. API 요청 (raise_for_status 사용)
# ===============================
try:
    response = session.get(BASE_URL, params=params, timeout=10)
    response.raise_for_status()
except requests.exceptions.HTTPError as e:
    raise RuntimeError(
        "[ERROR] HTTP 요청 실패\n"
        f"- 상태코드: {e.response.status_code}\n"
        f"- 응답내용:\n{e.response.text}"
    )
except requests.exceptions.RequestException as e:
    raise RuntimeError(
        "[ERROR] 네트워크 요청 중 예외 발생\n"
        f"- 상세: {str(e)}"
    )

# ===============================
# 6. XML 파싱
# ===============================
try:
    data = xmltodict.parse(response.text)
except Exception:
    raise RuntimeError(
        "[ERROR] XML 파싱 실패\n"
        "- 원인: API 응답이 XML 형식이 아님\n"
        "- 확인사항: Endpoint 및 ServiceKey 사용 여부\n\n"
        f"[응답 원문]\n{response.text}"
    )

# ===============================
# 7. API 헤더 에러 검사
# ===============================
header = data.get("response", {}).get("header", {})

result_code = header.get("resultCode")
result_msg = header.get("resultMsg")

if result_code and result_code != "00":
    raise RuntimeError(
        "[ERROR] KIPRIS API 응답 오류\n"
        f"- resultCode: {result_code}\n"
        f"- resultMsg : {result_msg}\n"
        "- 확인사항:\n"
        "  1) ServiceKey 유효성\n"
        "  2) 호출량 제한 초과 여부\n"
        "  3) 파라미터 유효성"
    )

# ===============================
# 8. body / items 검사
# ===============================
body = data.get("response", {}).get("body")

if not body:
    raise RuntimeError(
        "[ERROR] API 응답 body 없음\n"
        "- 원인: 검색 조건이 정상적으로 처리되지 않음\n"
        "- 확인사항: 요청 파라미터(applicant, patent, utility 등)"
    )

items = body.get("items", {}).get("item")

if not items:
    raise RuntimeError(
        "[ERROR] 검색 결과 없음\n"
        f"- 출원인명: {COMPANY_NAME}\n"
        "- 원인: applicant 파라미터가 무시되었을 가능성\n"
        "- 확인사항:\n"
        "  1) applicant → applicantName 또는 applicantNameList 필요 여부\n"
        "  2) API 명세서에서 지원 여부 확인"
    )

# ===============================
# 9. DataFrame 변환 및 컬럼 필터링
# ===============================
if isinstance(items, dict):
    items = [items]

full_df = pd.DataFrame(items)

target_columns = [
    "ipcNumber",          # IPC코드
    "applicationDate",    # 출원일자
    "astrtCont",          # 초록
    "applicationNumber",  # 출원번호
    "indexNo",            # 일련번호
    "registerStatus",     # 등록상태
    "inventionTitle",     # 발명의명칭
    "applicantName",      # 출원인
    "totalCount"          # 전체건수
]

available_columns = [c for c in target_columns if c in full_df.columns]
df = full_df[available_columns].copy()

# ===============================
# 10. 결과 확인
# ===============================
print(f"필터링된 컬럼 수: {len(df.columns)} / {len(target_columns)}")
df


In [ ]:
print("실제 API 응답 컬럼 리스트:", full_df.columns.tolist())

# 특허·실용 피인용문헌
- `StandardCitationApplicationNumber`, `ApplicationNumber`

In [ ]:
import os
import requests
import xmltodict
import pandas as pd
from dotenv import load_dotenv
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ===============================
# 1. 환경변수 로드
# ===============================
load_dotenv()
API_KEY = os.getenv("KIPRIS_API_KEY")

if not API_KEY:
    raise RuntimeError(
        "[ERROR] 환경변수 로드 실패\n"
        "- 원인: KIPRIS_API_KEY가 설정되어 있지 않습니다.\n"
        "- 확인사항: .env 파일에 KIPRIS_API_KEY가 존재하는지 확인하세요."
    )

# ===============================
# 2. Endpoint (CitingService)
# ===============================
BASE_URL = "http://plus.kipris.or.kr/openapi/rest/CitingService/citingInfo"

# ===============================
# 3. 검색 조건
# ===============================
APP_NUM = "1020230022879"

params = {
    "accessKey": API_KEY,
    "standardCitationApplicationNumber": APP_NUM
}

# ===============================
# 4. requests Session + Retry 설정
# ===============================
session = requests.Session()

retry_strategy = Retry(
    total=3,
    backoff_factor=1,                 # 1s → 2s → 4s
    status_forcelist=[500, 502, 503, 504],
    allowed_methods=["GET"]
)

adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("http://", adapter)
session.mount("https://", adapter)

# ===============================
# 5. API 요청 (raise_for_status)
# ===============================
try:
    response = session.get(BASE_URL, params=params, timeout=10)
    response.raise_for_status()
except requests.exceptions.HTTPError as e:
    raise RuntimeError(
        "[ERROR] HTTP 요청 실패\n"
        f"- 상태코드: {e.response.status_code}\n"
        f"- 응답내용:\n{e.response.text[:500]}"
    )
except requests.exceptions.RequestException as e:
    raise RuntimeError(
        "[ERROR] 네트워크 요청 중 예외 발생\n"
        f"- 상세: {str(e)}"
    )

# ===============================
# 6. XML 파싱
# ===============================
try:
    raw_data = xmltodict.parse(response.text)
except Exception:
    raise RuntimeError(
        "[ERROR] XML 파싱 실패\n"
        "- 원인: API 응답이 XML 형식이 아님\n"
        "- 확인사항:\n"
        "  1) Endpoint 경로 확인\n"
        "  2) accessKey 파라미터 이름 확인\n\n"
        f"[응답 원문 일부]\n{response.text[:500]}"
    )

# ===============================
# 7. API 헤더 에러 검사
# ===============================
res_root = raw_data.get("response", {})
header = res_root.get("header", {})

result_code = header.get("resultCode")
result_msg = header.get("resultMsg")

if result_code and result_code not in ["00", "0"]:
    raise RuntimeError(
        "[ERROR] KIPRIS API 응답 오류\n"
        f"- resultCode: {result_code}\n"
        f"- resultMsg : {result_msg}\n"
        "- 확인사항:\n"
        "  1) accessKey 유효성\n"
        "  2) 호출량 제한 초과 여부\n"
        "  3) 출원번호 형식 확인"
    )

# ===============================
# 8. body / citingInfo 파싱
# ===============================
body = res_root.get("body", {})
items_wrapper = body.get("items") if body else None

items = None
if items_wrapper:
    items = items_wrapper.get("citingInfo")

if items is None:
    raise RuntimeError(
        "[ERROR] 인용 정보 없음\n"
        f"- 출원번호: {APP_NUM}\n"
        "- 원인: 해당 특허에 인용/피인용 정보가 없음\n"
        "- 비고: API 호출은 정상 처리됨"
    )

# ===============================
# 9. DataFrame 변환 및 컬럼 필터링
# ===============================
if isinstance(items, dict):
    items = [items]

full_df = pd.DataFrame(items)

# 대소문자 무관 컬럼 매핑
target_map = {
    "standardcitationapplicationnumber": "StandardCitationApplicationNumber",
    "applicationnumber": "ApplicationNumber"
}

existing_cols = {col.lower(): col for col in full_df.columns}

final_cols = [
    existing_cols[k] for k in target_map
    if k in existing_cols
]

df = full_df[final_cols].copy() if final_cols else full_df

# ===============================
# 10. 결과 출력
# ===============================
if not df.empty:
    print(f"✅ 조회 성공! 데이터 건수: {len(df)}")
    print(df.head())
else:
    print("❌ 표시할 데이터가 없습니다.")
